# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

##    1. What test is appropriate for this problem? Does CLT apply?

Since there two independent samples, we can do a two-sample z-test. The sample sizes are large and have a binomial population, the 'b' and 'w' values are randomly assigned to the resumes, the normal conditions are met because number of call backs are greater than 10 and total population minus number of call backs is also greater than 10, therefore CLT does apply here.

##    2. What are the null and alternate hypotheses?



Ho = White and black sounding names receive the same call back rates
Ha = White and black sounding names receive different call back rates

##    3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
#number of call backs for black and white sounding names
call_b = sum(b.call)
call_w = sum(w.call)
#total number of resumes
total_w = float(len(w))
total_b = float(len(b))
#proportions of callbacks for each race
prop_w = call_w / total_w
prop_b = call_b / total_b
diff_prop = prop_w - prop_b
total_prop = (call_b + call_w)/(total_b + total_w)
print(f'Number of call backs for black sounding names: {call_b}')
print(f'Number of call backs for white sounding names: {call_w}')
print(f'Number of resumes for each set: {total_w}')
print(f'Proportion of callbacks for black names: {prop_b}')
print(f'Proportion of callbacks for white names: {prop_w}')

Number of call backs for black sounding names: 157.0
Number of call backs for white sounding names: 235.0
Number of resumes for each set: 2435.0
Proportion of callbacks for black names: 0.06447638603696099
Proportion of callbacks for white names: 0.09650924024640657


In [7]:
#since our null hypothesis is that there is no difference in callback rates between white and black
#sounding names, we can use the total proportion of callbacks to calculate variance
var_bandw = np.sqrt(total_prop * (1-total_prop) / total_w + total_prop * (1-total_prop) / total_b)

Use the z-score to compute P-value

In [8]:
z_score = diff_prop / var_bandw
p_value = stats.norm.sf(abs(z_score))
print(f"p-value: {p_value}")

p-value: 1.9919434187925383e-05


In [9]:
#margin of error
critical_value = 1.96
moe = critical_value*(w.call.std()/np.sqrt(total_w) + b.call.std()/np.sqrt(total_b))
print(f"Margin of error: {moe}")

Margin of error: 0.02148818910788651


In [10]:
#confidence interval
conf_int = np.array([diff_prop-moe, diff_prop+moe])
print(f"Confidence interval: {conf_int}")

Confidence interval: [0.01054467 0.05352104]


Boostrap method

In [11]:
#The difference in poroportions of call backs for each race could be due to chance
#one way to test this is using permutation
#We'll randomize the dataset and draw 10,000 samples into two arrays: black_names and white_names
#Then we'll compute the proportion of the call backs for each array and compare to that of real data.

def permutation_sample(data_1, data_2):
    """Generate a permutation sample from two data sets"""
    data = np.concatenate((data_1, data_2))
    permuted_data = np.random.permutation(data)
    perm_sample_1 = permuted_data[:len(data_1)]
    perm_sample_2 = permuted_data[len(data_1):]
    
    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generates multiple permutations replicates"""
    perm_replicates = np.empty(size)
    
    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates

def diff_of_props(data_1, data_2):
    """Difference in means of two arrays"""
    diff = np.sum(data_1)/len(data_1) - np.sum(data_2)/len(data_2)
    return diff

In [12]:
white_names = w.call
black_names = b.call
perm_replicates = draw_perm_reps(white_names, black_names, diff_of_props, 10_000)

In [13]:
p = np.sum(perm_replicates >= diff_prop) / len(perm_replicates)
p

0.0

Using both methods we obtained a p-value that is zero or close to zero. As such we must reject our null hypothesis that race has no implication on receiving a callback.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

##   4. Write a story describing the statistical significance in the context or the original problem.



We hear about racial descrimination everywhere and it is a big part of our lives today. When applying for jobs, race is the last "qualification" we want to worry about. Researchers examined the effect of having a black-sounding name vs white-sounding name by submitting almost 5000 resumes and see which ones would get callbacks. The results show that a white sounding name would receive a callback 50% more times than a black sounding name. This, of course, could be by chance, but our analysis shows that if this experiment was performed 10,000 times it is statistically impossible to achive such a large disrepency between callbacks just based on race alone. This shows that race has a statistical significance on receiving a callback from a potential employer.

##    5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Analysis performed above only indicates that there is a relationship between race and receiving a callback for a job application. The analysis does not weigh one variable more than the other and race alone may not be the only variable contributing to receiving a callback. Other factors can sway the decision even more such as experience, education, volunteering, and other factors. Each feature has to be evaluated separately to understand the full correlation. We can perform the same type of analysis on other features and determine which ones have correlations with receiving a callback.